In [ ]:
import re
from openpyxl import load_workbook # pip install --user openpyxl
from itertools import chain, islice, zip_longest
import pandas as pd
from IPython.display import display
from bidict import frozenbidict # pip install --user bidict
from functools import reduce, wraps, partial
import networkx as nx # pip install --user networkx
from pprint import pprint
import os.path
from libcrap.core import calcsave_or_load # pip install --user libcrap

In [ ]:
# path to "cable journal" excel file
# it's a MS Excel spreadsheet with a list of node connections
# in Lomonosov 2 cluster
# I am not allowed to share it.
SPREADSHEET_FILENAME = r'wire_journal_48_53.xlsx'

In [ ]:
# regex for parsing rack number and other numbers from cells
# with switch names in the spreadsheet
switch_regex = re.compile(
    r"""
    КГК\.       # literally match what is written here
    (?P<rack>\d+)\.        # rack number is one or more digits, followed by dot
    (?P<second_number>\d+)\.            # then goes another non-negative integer followed by dot
    (?P<last_number>\d+)            # and another integer of the same form
    """,
    re.VERBOSE)

assert switch_regex.match("КГК.63.2.4").groups() == ("63", "2", "4")

## Parse data from the spreadsheet using openpyxl

In [ ]:
def get_column_name(column):
    """Takes column as tuple as argument and returns
    its name as string"""
    return column[0].column

In [ ]:
def extract_columns(worksheet, column_names):
    """
    parameters:
        worksheet -- worksheet
        column_names -- list of strings, for example
            ['A', 'C', 'E']
    returns:
        list of columns, where every column is represented
        as a tuple"""
    all_columns = worksheet.columns
    extracted_columns = [col for col in all_columns
                         if get_column_name(col) in column_names]
    assert len(extracted_columns) == len(column_names)
    return extracted_columns

In [ ]:
def columns_to_tuples(columns):
    """parameters:
        columns -- columns as a tuple/list of tuples
    returns:
        list of lists/tuples, each one represents a row"""
    return [[cell.value for cell in row] for row in zip(*columns)]

In [ ]:
def parse_switch_pairs(workbook):
    """Parse openpyxl workbook and extract a list of
    pairs of switches. Pair (A, B) means that swithes A
    and B are connected.
    Returns list of pairs of strings."""
    return list(chain(*[columns_to_tuples(extract_columns(worksheet, ['C', 'K']))
            for worksheet in workbook]))

In [ ]:
# Lomonosov2's racks are grouped into pairs
# Switches in the same rack or pair of racks are connected with copper wires
# Switches in different pairs of racks are connected with optic cable
RACK_PAIRS = ((48, 49), (50, 51), (52, 53))

In [ ]:
def get_rack(switch_name):
    """Determines rack number from switch name"""
    return int(switch_regex.match(switch_name).group('rack'))

assert get_rack('КГК.48.0.1') == 48

In [ ]:
def determine_material_between_switches(rack1, rack2):
    """Switches have different material between them.
    See comment about RACK_PAIRS.
    
    This function determines cable material between two switches
    by using their rack numbers and returns it as string"""
    racks = (rack1, rack2)
    if any(
            all(rack in rack_pair for rack in racks)
            for rack_pair in RACK_PAIRS):
        # they are in the same pair of racks
        return 'copper'
    return 'optic'

In [ ]:
def get_cable_material_from_row(row):
    rack1 = row.loc["switch1_rack"]
    rack2 = row.loc["switch2_rack"]
    return determine_material_between_switches(rack1, rack2)

In [ ]:
def get_second_number(switch_name):
    return switch_regex.match(switch_name).groups()[1]

In [ ]:
def get_third_number(switch_name):
    return switch_regex.match(switch_name).groups()[2]

In [ ]:
workbook = load_workbook(SPREADSHEET_FILENAME, data_only=True)

In [ ]:
switch_pairs = parse_switch_pairs(workbook)

In [ ]:
switches = pd.DataFrame({
    "name": sorted(list(frozenset(chain(*switch_pairs))))
})
switches["rack_number"] = switches["name"].apply(get_rack)
switches["second_number"] = switches["name"].apply(get_second_number)
switches["third_number"] = switches["name"].apply(get_third_number)

In [ ]:
switches

In [ ]:
switch_to_switch_connections = (
    # add rack numbers for switch1 column
    pd.merge(
        # convert list of pairs to DataFrame
        pd.DataFrame.from_records(switch_pairs, columns=["switch1", "switch2"]),
        switches,
        left_on=["switch1"], right_on=["name"])
    .rename(columns={"rack_number": "switch1_rack"})
    [["switch1", "switch2", "switch1_rack"]]
    
    # add rack numbers for switch2 column
    .merge(
        switches,
        left_on=["switch2"], right_on=["name"])
    .rename(columns={"rack_number": "switch2_rack"})
    [["switch1", "switch2", "switch1_rack", "switch2_rack"]]
    
    # add cable type
    .assign(cable_type=lambda df: df.apply(
        lambda row: determine_material_between_switches(row["switch1_rack"], row["switch2_rack"]),
        axis=1
    ))
    .drop(["switch1_rack", "switch2_rack"], axis=1)
)

In [ ]:
switch_to_switch_connections.iloc[510:513]

Now let's build table with connections between switches and computational
nodes connected directly to them.

In [ ]:
def get_matching_computational_nodes(switch):
    """params:
        switch -- string, name of the switch
    returns:
        list of strings which are names of computational
        nodes connected to this switch"""
    get_thingie = switch_regex.match(switch).group
    return [
        'n{0}{1}{2:02d}'.format(
            get_thingie('rack'),
            get_thingie('second_number'),
            (int(get_thingie('last_number')) - 1) * 8 + i
        )
        for i in range(1, 9)
    ]

assert get_matching_computational_nodes("КГК.48.2.3") == [
    'n48217',
    'n48218',
    'n48219',
    'n48220',
    'n48221',
    'n48222',
    'n48223',
    'n48224'
]

In [ ]:
comp_node_to_switch_connects = pd.concat(
    (pd.DataFrame.from_dict({
        "computational_node": get_matching_computational_nodes(switch),
        "switch": switch})
    for switch in switches["name"]),
    ignore_index=True
)
assert len(comp_node_to_switch_connects) == 1536

In [ ]:
comp_node_to_switch_connects

Now let's build table with all graph edges, that is a single table
with switch-switch and switch-comp_node connections.

In [ ]:
edges = pd.concat([
    switch_to_switch_connections
        .rename(columns={
            "switch1": "node1",
            "switch2": "node2",
            "cable_type": "connection_type"
        }),
    comp_node_to_switch_connects
        .rename(columns={
            "computational_node": "node1",
            "switch": "node2"
        })
        .assign(connection_type="backplane")
])
edges["connection_type"] = edges["connection_type"].astype("category")

In [ ]:
edges

In [ ]:
edges.info()

Now let's make a table with all nodes and all their properties

In [ ]:
nodes = pd.concat([
    switches[["name"]]
        .assign(type_="switch"),
    comp_node_to_switch_connects[["computational_node"]]
        .rename(columns={"computational_node": "name"})
        .assign(type_="computational")
])
nodes["type_"] = nodes["type_"].astype("category")

In [ ]:
nodes

Now I want to make copies of these 2 tables with all nodes' and edges' properties
replaced with numbers. Categorical values (like computational, switch) or
(copper, backplane, optic) should be translated to numbers like 0, 1, 2.

Then I will calculate shortest path for every pair of nodes (we will assume that
every edge has the same weight) and I will write down a sequence of numbers which
will mean the sequence of properties of all nodes and edges in the shortest
path.

E.g. `(comp_node) --backplane-- (switch) --optic-- (switch) --backplane-- (comp_node)`
might get translated to **1,2,0,1,0,2,1**.

In [ ]:
def map_categorial_sequence_to_numbers(sequence):
    """Takes a sequence of values as input.
    Maps each unique value to an integer.
    
    Returns a tuple: (
      the same sequence as numpy.ndarray of resulting integers
      ,
      mapping
    )"""
    new_sequence, unique_values = pd.factorize(sequence, sort=True)
    return new_sequence, frozenbidict(enumerate(unique_values)).inv

In [ ]:
def map_categorial_df_to_numbers(df, columns):
    """Arguments:
    df -- pandas.DataFrame in which we will transform columns (not inplace)
    columns -- sequence of column names
    
    Returns a tuple (new_data_frame, mappings)
    new_data_frame is a DataFrame with `columns` values replaced with
      what they were mapped to
    mappings - a sequence of frozenbidicts with mappings of values,
    one for each column. These bidicts are ordered the same way
    as the argument `columns`."""
    assert type(df) == pd.DataFrame
    mappings = []
    new_df = df.copy()
    for column in columns:
        new_df[column], mapping = map_categorial_sequence_to_numbers(df[column])
        mappings.append(mapping)
    return new_df, mappings

In [ ]:
def test_map_categorial_df_to_numbers():
    df = pd.DataFrame.from_records(
        [
            ("john", "male", "high"),
            ("lisa", "female", "low"),
            ("jack", "male", "low"),
            ("anna", "female", "medium"),
            ("boris", "male", "medium")
        ],
        columns=["name", "sex", "height"]
    )
    new_df, mappings = map_categorial_df_to_numbers(df, ["height", "sex"])
    display(mappings)
    display(new_df)

#test_map_categorial_df_to_numbers()

In [ ]:
nodes_with_numerical_properties, nodes_mappings = map_categorial_df_to_numbers(
    nodes, ["type_"]
)

In [ ]:
edges_with_numerical_properties, edges_mappings = map_categorial_df_to_numbers(
    edges, ["connection_type"]
)

In [ ]:
display(nodes_mappings)
display(nodes_with_numerical_properties)

In [ ]:
display(edges_mappings)
display(edges_with_numerical_properties)

Now I will create NetworkX graph from these 2 tables

In [ ]:
topology = nx.Graph()
for index, row in nodes_with_numerical_properties.iterrows():
    node_name = row["name"]
    attributes = row.drop(["name"]).to_dict()
    topology.add_node(node_name, attr_dict=attributes)

In [ ]:
for index, row in edges_with_numerical_properties.iterrows():
    node1_name = row["node1"]
    node2_name = row["node2"]
    attributes = row.drop(["node1", "node2"]).to_dict()
    topology.add_edge(node1_name, node2_name, attr_dict=attributes)

Now I will find shortest path for every pair of nodes

In [ ]:
# this is a pandas Series of all computational nodes
comp_nodes = nodes[nodes["type_"] =="computational"]["name"]
display(comp_nodes.head(2))
display(comp_nodes.tail(2))

In [ ]:
pd_diskcache = partial(calcsave_or_load, load_func=pd.read_pickle, save_func=pd.to_pickle)

In [ ]:
@pd_diskcache("paths.pkl")
def find_comp_to_comp_shortest_paths(topology, comp_nodes):
    paths_ugly = nx.all_pairs_shortest_path(topology)
    # calculates shortest paths and stores them in a dict of dicts
    
    # build a table with all computational node pairs
    # they are not duplicated
    # if there is ("n48001", "n49419") then there is no ("n49419", "n48001") pair
    comp_node_pairs = pd.DataFrame.from_records(
        chain.from_iterable(
            [(node1, node2) for node2 in comp_nodes.iloc[index+1:]]
            for (index, node1) in comp_nodes.iteritems()
        ),
        columns=["node1", "node2"]
    )

    # write shortest paths to this table
    comp_node_pairs["shortest_path"] = comp_node_pairs.apply(
        lambda row: paths_ugly[row.loc["node1"]][row.loc["node2"]],
        axis=1
    )
    return comp_node_pairs

In [ ]:
# shortest paths between all computational nodes
paths = find_comp_to_comp_shortest_paths(topology, comp_nodes)

In [ ]:
display(paths.iloc[400:404])

Now let's convert these lists of nodes to lists of properties.
Edges have properties, nodes have properties - let's make lists
of those.

In [ ]:
def get_node_properties(topology, node):
    """Returns node properties as a dict."""
    return topology.node[node]

assert get_node_properties(topology, "КГК.48.0.3") == {"type_": 1}

In [ ]:
def get_edge_properties(topology, node1, node2):
    """Returns edge properties as a dict."""
    return topology.edge[node1][node2]

def test_get_edge_properties():
    correct_result = {"connection_type": 0}
    result1 = get_edge_properties(topology, "КГК.48.0.3", "n48022")
    result2 = get_edge_properties(topology, "n48022", "КГК.48.0.3")
    assert result1 == correct_result == result2

test_get_edge_properties()

In [ ]:
def interleave(it1, it2):
    return (
        item for item
        in chain.from_iterable(zip_longest(it1, it2))
        if item is not None)

In [ ]:
def path_to_properties(topology, path):
    # path[0] is a node
    property0 = [get_node_properties(topology, path[0])]
    
    nodes_properties = (get_node_properties(topology, node) for node in path)
    
    edges_properties = (get_edge_properties(topology, node1, node2)
                        for (node1, node2) in zip(path[:-1], path[1:]))
    return list(interleave(nodes_properties, edges_properties))

In [ ]:
@pd_diskcache("paths_properties.pkl")
def calc_paths_properties(topology, paths):
    paths_properties = paths.copy()
    paths_properties["shortest_path_properties"] = \
        paths["shortest_path"].apply(partial(path_to_properties, topology))
    return paths_properties

In [ ]:
%%timeit -r 1
paths_properties = paths.copy()
paths_properties["shortest_path_properties"] = \
    paths["shortest_path"].apply(partial(path_to_properties, topology))

In [ ]:
display(paths_properties.iloc[0]["shortest_path"])
display(paths_properties.iloc[0]["shortest_path_properties"])